# Supermarket Transaction Analysis in SQL 

The checkout line is a familiar part of the retail experience. Any retailer needs to understand their checkout data to answer important questions like "do people spend more with card or cash?", "how many items do people buy at different times?", and "are self-service checkouts quicker than those with a cashier?". 

Analyze [transaction data from a Polish Supermarket](https://www.mdpi.com/2306-5729/4/2/67/htm) to answer financial and logistical questions.

A financial analysis of point of sale transactions can have many purposes. A non-exhaustive list is:
- increase sales by identifying what matters to the customer, is it promotions? on what items? is it opening hours? prices of particular items? and so on;
- increasing productivity of stores and operators by identifying factors that result in Lower processing time per item or basket size, accurate predictions for labour scheduling, adequate level of inventory, fewer scanning errors at checkout, and so on;
- predict sales in the short, medium and long term to inform decisions on investments in fixed assets and human resources, financing, go-to-market invesments, etc.

##  1: Exploring the data

Selecting the first few rows from each table.

In [ ]:
SELECT *
FROM 'pos_operator_logs.csv'
LIMIT 100;

,id,Workstation_Group_ID,Workstation_ID,begin_date_time,operator_id
0,1,8,19,2019-02-13 05:37:55+00:00,269
1,2,8,18,2019-02-13 05:37:55+00:00,268
2,3,8,17,2019-02-13 05:38:43+00:00,267
3,4,1,4,2019-02-13 07:01:26+00:00,332
4,5,1,7,2019-02-13 07:01:57+00:00,10
...,...,...,...,...,...
95,96,1,20,2019-02-13 10:13:59+00:00,328
96,97,1,21,2019-02-13 10:17:43+00:00,338
97,98,1,20,2019-02-13 10:24:06+00:00,328
98,99,1,20,2019-02-13 10:24:24+00:00,328


In [2]:
SELECT *
FROM 'pos_transactions.csv'
LIMIT 100;

,id,WorkstationGroupID,begin_date_time,end_date_time,OperatorID,basket_size,t_cash,t_card,amount
0,1,1,2017-12-07 14:23:23+00:00,2017-12-07 14:24:36+00:00,101,23,True,False,112.71
1,2,1,2017-12-07 14:25:09+00:00,2017-12-07 14:27:00+00:00,101,29,True,False,54.76
2,3,1,2017-12-07 14:27:28+00:00,2017-12-07 14:27:48+00:00,101,3,True,False,14.77
3,4,1,2017-12-07 14:28:04+00:00,2017-12-07 14:28:29+00:00,101,12,True,False,37.88
4,5,1,2017-12-07 14:29:40+00:00,2017-12-07 14:30:32+00:00,101,7,True,False,115.34
...,...,...,...,...,...,...,...,...,...
95,96,1,2017-12-07 17:07:48+00:00,2017-12-07 17:08:39+00:00,101,9,True,False,32.54
96,97,1,2017-12-07 17:10:51+00:00,2017-12-07 17:11:33+00:00,101,9,True,False,26.59
97,98,1,2017-12-07 17:11:47+00:00,2017-12-07 17:12:37+00:00,101,20,True,False,104.82
98,99,1,2017-12-07 17:12:51+00:00,2017-12-07 17:13:58+00:00,101,12,False,True,67.62


## 2: Do more people make transactions by card or by cash?

Understanding how people pay is crucial for shop logistics like deciding what kind of checkout equipment to buy.


Counting how many transactions were processed by cash (the case when `t_cash` is true) and by card  (the case when `t_card` is true) using the `pos_transactions` table.

In [4]:
SELECT 
COUNT (CASE WHEN t_cash THEN 1 END) AS cash_transaction,
COUNT (CASE WHEN t_card THEN 1 END) AS card_transaction

FROM 'pos_transactions.csv'

,cash_transaction,card_transaction
0,84487,78246


## 3: Do people spend more per transaction when using cash or card?

Knowing how much people spend per transaction using different payment methods is helpful for deciding on security arrangments, and marketing to encourage shoppers to use different payment methods.

Finding the average per transaction type when a transaction is  card, or cash.

Calculating the mean (average) amount spent per transaction for cash and for card.

In [7]:
SELECT t_cash,
t_card,
AVG(amount) AS avg_transaction
FROM 'pos_transactions.csv'
GROUP BY t_cash, t_card

,t_cash,t_card,avg_transaction
0,False,False,70.570113
1,True,True,166.576144
2,True,False,58.718519
3,False,True,86.839370


## 4: Exploring Sunday Trading

Regulations were introduced in Poland in 2018 banned shopping on some Sundays. It was generally two Sundays per month in 2018 and three Sundays per month in 2019. Supermarkets have reacted by extending the working hours on Fridays and Saturdays, while customers had to adapt to the changes in opening hours. **The dataset contains two working Sundays (24 February, 31 March) and two non-working Sundays (17 February, 7 April).**


The week number of the dates where trading occured on a Sunday:

In [10]:
SELECT
DATE_PART('week', end_date_time::DATE) AS week_num,
end_date_time::DATE AS end_date

FROM 'pos_transactions.csv'
WHERE EXTRACT(year FROM end_date_time) >= 2019
	AND end_date IN ('2019-02-17','2019-02-24','2019-03-31','2019-04-07')
GROUP BY end_date
ORDER BY end_date

,week_num,end_date
0,8,2019-02-24 00:00:00+00:00
1,13,2019-03-31 00:00:00+00:00


The last two dates do not appear because they were non-working Sundays. 

So the Weeks with with working Sundays are 8 and 13. And the weeks with non-working Sundays are 7 and 14. 

## 5: Analysis of daily trends

Understanding which days are busiest is important for deciding how many staff to schedule shifts for, as well as helping to plan financially.

- Selecting all transactions of 2019 and find out the daily sales amount, the daily average amount per transaction and the median average amount per transaction and group by week

Do we see different trends on different days of the week? 

In [7]:
SELECT
	EXTRACT(week FROM end_date_time) AS week,
	end_date_time::DATE AS end_date,
	COUNT(id) AS total_transactions,
	SUM (amount) AS daily_sales_amt,
	AVG(amount) AS daily_avg_transaction,


FROM 'pos_transactions.csv'
WHERE EXTRACT(year FROM end_date_time) >= 2019
GROUP BY week, end_date
ORDER BY week, end_date

,week,end_date,total_transactions,daily_sales_amt,daily_avg_transaction
0,7,2019-02-13 00:00:00+00:00,3163,189112.49,59.788963
1,7,2019-02-14 00:00:00+00:00,4404,302242.45,68.629076
2,7,2019-02-15 00:00:00+00:00,4031,319112.44,79.164584
3,7,2019-02-16 00:00:00+00:00,4669,405580.79,86.866736
4,8,2019-02-18 00:00:00+00:00,3270,193133.29,59.062168
5,8,2019-02-19 00:00:00+00:00,3025,170242.42,56.278486
6,8,2019-02-20 00:00:00+00:00,3045,181630.10,59.648637
7,8,2019-02-21 00:00:00+00:00,4368,298612.75,68.363725
8,8,2019-02-22 00:00:00+00:00,4411,330428.78,74.910175
9,8,2019-02-23 00:00:00+00:00,4367,369412.97,84.591933


Adding the average and median `basket_size` per day in 2019. 



In [ ]:
SELECT
	EXTRACT(week FROM end_date_time) AS week,
	EXTRACT(day FROM end_date_time) AS day,
	EXTRACT(dow FROM end_date_time) AS dow,
	end_date_time::DATE AS end_date,
	COUNT(id) AS total_transactions,
	ROUND(SUM (amount),2) AS daily_sales_amt,
	ROUND(AVG(amount),2) AS daily_avg_transaction,
	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY amount) AS median_sale_amt,
	ROUND(AVG(basket_size),2) AS avg_basket_size,
	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY basket_size) AS median_basket_size

FROM 'pos_transactions.csv'
WHERE EXTRACT(year FROM end_date_time) >= 2019
GROUP BY week, day, dow, end_date
ORDER BY week, day, end_date

,week,day,end_date,total_transactions,daily_sales_amt,daily_avg_transaction,median_sale_amt,avg_basket_size,median_basket_size
0,7,13,2019-02-13 00:00:00+00:00,3163,189112.49,59.79,37.580,12.92,9.0
1,7,14,2019-02-14 00:00:00+00:00,4404,302242.45,68.63,46.285,14.17,10.0
2,7,15,2019-02-15 00:00:00+00:00,4031,319112.44,79.16,50.900,17.18,12.0
3,7,16,2019-02-16 00:00:00+00:00,4669,405580.79,86.87,57.600,19.73,14.0
4,8,18,2019-02-18 00:00:00+00:00,3270,193133.29,59.06,37.420,13.81,10.0
5,8,19,2019-02-19 00:00:00+00:00,3025,170242.42,56.28,37.530,13.43,9.0
6,8,20,2019-02-20 00:00:00+00:00,3045,181630.10,59.65,36.560,13.59,9.0
7,8,21,2019-02-21 00:00:00+00:00,4368,298612.75,68.36,45.865,16.49,12.0
8,8,22,2019-02-22 00:00:00+00:00,4411,330428.78,74.91,49.060,17.74,12.0
9,8,23,2019-02-23 00:00:00+00:00,4367,369412.97,84.59,56.370,19.81,14.0


## 6: Should the supermarket open on Sundays?

The two Sundays are significantly lower in volume of transactions than all other days. 
Also, Thursdays, Fridays and Saturdays are approximately 25% higher in number of transactions and 40% higher in sales revenue than Mondays, Tuesdays and Wednesdays.

The full week data is available only for week 8  and week 14.


### Instructions

Calculate the total weekly sales amount for week 8 and 14 in 2019.

In [ ]:
SELECT
	EXTRACT(week FROM end_date_time) AS week,
	ROUND(SUM(amount), 2) AS total_weekly_sales_amt

FROM 'pos_transactions.csv'

WHERE EXTRACT(year FROM end_date_time) = 2019 
  AND EXTRACT(week FROM end_date_time) IN (8, 14)
GROUP BY week
ORDER BY week

,week,total_weekly_sales_amt
0,8,1686485.72
1,14,1690989.87


## 7: How much does labor cost?

In [ ]:
SELECT
	EXTRACT(week FROM begin_date_time) AS week,
	EXTRACT(day FROM begin_date_time) AS day,
	EXTRACT(dow FROM begin_date_time) AS dow,
	begin_date_time::DATE AS begin_date,
	COUNT(DISTINCT operator_id) AS total_operators

FROM 'pos_operator_logs.csv'
WHERE EXTRACT(year FROM begin_date_time) = 2019
GROUP BY week, day, dow, begin_date
ORDER BY week, day, begin_date

,week,day,dow,begin_date,total_operators
0,7,13,3,2019-02-13 00:00:00+00:00,20
1,7,14,4,2019-02-14 00:00:00+00:00,26
2,7,15,5,2019-02-15 00:00:00+00:00,24
3,7,16,6,2019-02-16 00:00:00+00:00,28
4,8,18,1,2019-02-18 00:00:00+00:00,20
5,8,19,2,2019-02-19 00:00:00+00:00,21
6,8,20,3,2019-02-20 00:00:00+00:00,19
7,8,21,4,2019-02-21 00:00:00+00:00,27
8,8,22,5,2019-02-22 00:00:00+00:00,26
9,8,23,6,2019-02-23 00:00:00+00:00,25


The stores had 18 employees working on Sunday 24 February and 20 employees working on Sunday 31 March. 
I would then look into the hours per shift and the cost per hour to calcute the average savings per Sunday, as well as estimate all other costs of keeping stores open and operational (e.g. energy costs). On the other hand stores may have additional costs for not opening one day, for example for short shelf life goods that may go unsold. 